# **Model 3 - ABSA (Aspect-Based Sentiment Analysis)**

## **Load Non Spam Dataset**

In [1]:
import os
import pandas as pd
import numpy as np
import json
from tqdm.auto import tqdm

In [43]:
df = pd.read_csv(
    "https://drive.google.com/uc?id=1MxEAAtv27aH77ZcnwsyUb27al0l1QWej",
    encoding="utf-8"
)

TEXT_COL = "text_norm"

df = df[df["is_spam"] == 0].copy()
df = df[df[TEXT_COL].notna() & (df[TEXT_COL].str.strip() != "")].copy()
df = df.reset_index(drop=True)

selected_cols = [
    "text",
    "rating",
    "category",
    "product_name",
    "product_id",
    "sold",
    "shop_id",
    "product_url",
    "text_clean",
    "text_norm"
]
df = df[selected_cols]

print("Jumlah review CLEAN:", len(df))

Jumlah review CLEAN: 28968


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28968 entries, 0 to 28967
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          28968 non-null  object
 1   rating        28968 non-null  int64 
 2   category      28968 non-null  object
 3   product_name  28968 non-null  object
 4   product_id    28968 non-null  int64 
 5   sold          28968 non-null  int64 
 6   shop_id       28968 non-null  int64 
 7   product_url   28968 non-null  object
 8   text_clean    28968 non-null  object
 9   text_norm     28968 non-null  object
dtypes: int64(4), object(6)
memory usage: 2.2+ MB


In [45]:
df.head()

,text,rating,category,product_name,product_id,sold,shop_id,product_url,text_clean,text_norm
0,Paket rapi...mantap....cepat....sampe ke tujuan,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...,packing rapi mantap cepat sampai ke tujuan,packing rapi mantap cepat sampai ke tujuan
1,ya saya puas dgn barangnya,5,pertukangan,ALAT STAPLES TEMBAK &#40;AIR NAILER GUN&#41; O...,102279869,5,771395,https://www.tokopedia.com/kamarmesin/alat-stap...,ya saya puas dengan barang,ya saya puas dengan barang
2,Responya luar biasa b mantap,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,respon luar biasa b mantap,respon luar biasa b mantap
3,"seller top, pengiriman cepat barang oke",5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,seller bagus pengiriman cepat barang oke,seller bagus pengiriman cepat barang oke
4,pengiriman cepat seller top,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,pengiriman cepat seller bagus,pengiriman cepat seller bagus


## **Split Train and Test Data**

In [46]:
df_test = df.iloc[:1000].copy()
df_train = df.iloc[1000:].copy()

print("Shape of df_test:", df_test.shape)
print("Head of df_test:")
display(df_test.head())

print("\nShape of df_train:", df_train.shape)
print("Head of df_train:")
display(df_train.head())

Shape of df_test: (1000, 10)
Head of df_test:


,text,rating,category,product_name,product_id,sold,shop_id,product_url,text_clean,text_norm
0,Paket rapi...mantap....cepat....sampe ke tujuan,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...,packing rapi mantap cepat sampai ke tujuan,packing rapi mantap cepat sampai ke tujuan
1,ya saya puas dgn barangnya,5,pertukangan,ALAT STAPLES TEMBAK &#40;AIR NAILER GUN&#41; O...,102279869,5,771395,https://www.tokopedia.com/kamarmesin/alat-stap...,ya saya puas dengan barang,ya saya puas dengan barang
2,Responya luar biasa b mantap,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,respon luar biasa b mantap,respon luar biasa b mantap
3,"seller top, pengiriman cepat barang oke",5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,seller bagus pengiriman cepat barang oke,seller bagus pengiriman cepat barang oke
4,pengiriman cepat seller top,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,pengiriman cepat seller bagus,pengiriman cepat seller bagus



Shape of df_train: (27968, 10)
Head of df_train:


,text,rating,category,product_name,product_id,sold,shop_id,product_url,text_clean,text_norm
1000,Sesuai dengan yg diharapkan pengiriman dan res...,5,pertukangan,Hekter Tembak / Staples / Guntacker,11888855,175,88752,https://www.tokopedia.com/suryanusantara/hekte...,sesuai dengan yang diharapkan pengiriman denga...,sesuai dengan yang diharapkan pengiriman denga...
1001,"Barang sudah sampai dan diterima dengan baik, ...",5,pertukangan,Hekter Tembak / Staples / Guntacker,11888855,175,88752,https://www.tokopedia.com/suryanusantara/hekte...,barang susah sampai dengan diterima dengan bai...,barang susah sampai dengan diterima dengan bai...
1002,"Staples nya cukup bagus, terima kasih",5,pertukangan,Hekter Tembak / Staples / Guntacker,11888855,175,88752,https://www.tokopedia.com/suryanusantara/hekte...,staples nya cukup bagus terima kasih kasih,staples nya cukup bagus terima kasih kasih
1003,barang bagus kondisi ok,5,pertukangan,Hekter Tembak / Staples / Guntacker,11888855,175,88752,https://www.tokopedia.com/suryanusantara/hekte...,barang bagus kondisi oke,barang bagus kondisi oke
1004,barangnya sudah sampai \nbarang bagus,4,pertukangan,Hekter Tembak / Staples / Guntacker,11888855,175,88752,https://www.tokopedia.com/suryanusantara/hekte...,barang susah sampai barang bagus,barang susah sampai barang bagus


## **Labelling**

In [47]:
ASPECT_LABELS = [
    "Kualitas Barang",
    "Pelayanan Penjual",
    "Kemasan Barang",
    "Harga Barang",
    "Sesuai Deskripsi",
    "Pengiriman",
    "Lainnya"
]

In [48]:
# NUM_SAMPLES = 500

# sample_df = df_train.sample(NUM_SAMPLES, random_state=42).copy()

# sample_df = sample_df.reset_index(drop=False).rename(columns={"index": "review_id"})

# sample_df["absa_labels"] = ""

# cols_prioritas = ["review_id", TEXT_COL, "category", "product_name"]
# cols_prioritas = [c for c in cols_prioritas if c in sample_df.columns]

# template_df = sample_df[cols_prioritas + ["absa_labels"]]

# template_path = "absa_labeled_sample.csv"
# template_df.to_csv(template_path, index=False)

# print(f"Template labelling disimpan ke: {template_path}")
# template_df.head()

In [49]:
import ast

def parse_absa_cell(cell):
    if pd.isna(cell):
        return []
    cell = str(cell).strip()
    if cell == "":
        return []
    try:
        return json.loads(cell)
    except Exception:
        try:
            return ast.literal_eval(cell)
        except Exception:
            return []

In [50]:
df_lab = pd.read_csv(
    "https://drive.google.com/uc?id=1Np-Vjg6g0Qcj4VHXg2EejJ2ciDT-9Iw9",
    encoding="latin-1"
)

df_lab["absa_labels"] = df_lab["absa_labels"].apply(parse_absa_cell)

print("Jumlah review ber-label:", len(df_lab))
df_lab.head()

Jumlah review ber-label: 500


,review_id,text_norm,category,product_name,absa_labels
0,16782,pelayanan cepat barang bagus bagus seller,elektronik,Gamepad single Usb M-Tech/stick laptop/stick p...,"[{'aspect': 'Pelayanan Penjual', 'opinion_span..."
1,22891,mantap bos pengiriman juga cepat,elektronik,TINTA / CATRIDGE HP 802 COLOR ORIGINAL 100%,"[{'aspect': 'Pengiriman', 'opinion_span': 'pen..."
2,19304,oke terima kasih kasih keyboardnya susah diter...,elektronik,Keyboard Toshiba Satellite C600 C600D C605 C63...,"[{'aspect': 'Lainnya', 'opinion_span': 'oke te..."
3,11212,tidak iklan kartu garansi kaca dari barang res...,fashion,Jam Tangan Casio LTP-1275SG-7B,"[{'aspect': 'Kualitas Barang', 'opinion_span':..."
4,18415,barang dh sampai terima kasih semoga barang awet,elektronik,Speaker komputer/speaker laptop/speaker aktif/...,"[{'aspect': 'Pengiriman', 'opinion_span': 'bar..."


### **Inject Index**

In [51]:
def find_span_indices(row):
    text = row[TEXT_COL].lower()
    labels = row['absa_labels']

    new_labels = []
    for label in labels:
        span = label['opinion_span'].lower()
        if not span: continue

        start_idx = text.find(span)

        if start_idx != -1:
            end_idx = start_idx + len(span)
            label['start'] = start_idx
            label['end'] = end_idx
            new_labels.append(label)
        else:
            print(f"Span '{span}' not found in text: {text}")
            pass

    return new_labels

In [52]:
df_lab['absa_labels_indexed'] = df_lab.apply(find_span_indices, axis=1)

print(f"Total Labels Awal: {df_lab['absa_labels'].apply(len).sum()}")
print(f"Total Labels Indexed: {df_lab['absa_labels_indexed'].apply(len).sum()}")

df_clean_train = df_lab[df_lab['absa_labels_indexed'].map(len) > 0].copy()
print(f"Data siap training: {len(df_clean_train)}")

Span 'pas untuk kayu yang lunak' not found in text: produk bagus susah dicoba untuk kayu yang lunak pas untuk kayu yang kemas kurang pas seller bagus cepat rekomen
Span 'lumayan tapi biasa kepake' not found in text: lumayan meski murah tapi biasa kepake untuk multy media main men resolum arkeos multi screen pada mac book i7 tengkyu multy komputer
Span 'service cepat' not found in text: bagus kualitas oke sesuai harapan harga bersaing cepat service shipping kalau biasa ditambahkan option ekspedisi selain jne makin bagus deh packing rapi rekomen seller terima kasih ya warung
Span 'shipping cepat' not found in text: bagus kualitas oke sesuai harapan harga bersaing cepat service shipping kalau biasa ditambahkan option ekspedisi selain jne makin bagus deh packing rapi rekomen seller terima kasih ya warung
Span 'ukuran berbeda2' not found in text: 1 packing dapat 3 kaos kaki tapi ukurannya berbeda2 iklan yang kecil iklan yang besar warna juga berbeda dengan di foto
Span 'kabel ... berfungsi 

## **BIO Tagging**

In [53]:
unique_aspects = ASPECT_LABELS
label_list = ["O"]
for asp in unique_aspects:
    code = asp.replace(" ", "_")
    label_list.append(f"B-{code}")
    label_list.append(f"I-{code}")

label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for i, l in enumerate(label_list)}

print(f"Total Tags: {len(label_list)}")
print("Tags:", label_list)

Total Tags: 15
Tags: ['O', 'B-Kualitas_Barang', 'I-Kualitas_Barang', 'B-Pelayanan_Penjual', 'I-Pelayanan_Penjual', 'B-Kemasan_Barang', 'I-Kemasan_Barang', 'B-Harga_Barang', 'I-Harga_Barang', 'B-Sesuai_Deskripsi', 'I-Sesuai_Deskripsi', 'B-Pengiriman', 'I-Pengiriman', 'B-Lainnya', 'I-Lainnya']


## **Tokenisasi & Alignment**

In [54]:
from transformers import AutoTokenizer

MODEL_CHECKPOINT = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[TEXT_COL],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=False
    )

    labels = []

    for i, text in enumerate(examples[TEXT_COL]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        original_labels = examples['absa_labels_indexed'][i]

        # Character Span Mapping
        # Inisialisasi list label per token dengan 'O' (Id 0)
        label_ids = [0] * len(tokenized_inputs["input_ids"][i])

        # Map setiap span karakter ke token
        for lab in original_labels:
            start_char = lab['start']
            end_char = lab['end']
            category_code = lab['aspect'].replace(" ", "_")
            b_tag = label2id.get(f"B-{category_code}", 0)
            i_tag = label2id.get(f"I-{category_code}", 0)

            # Cari token mana yang mencakup karakter start_char sampai end_char
            token_start_index = tokenized_inputs.char_to_token(i, start_char)
            token_end_index = tokenized_inputs.char_to_token(i, end_char - 1)

            # Handling jika token ada di luar max_length
            if token_start_index is None or token_end_index is None:
                continue

            # Set Label B-Tag
            label_ids[token_start_index] = b_tag

            # Set Label I-Tag untuk sisa token di span tersebut
            if token_end_index > token_start_index:
                for k in range(token_start_index + 1, token_end_index + 1):
                    label_ids[k] = i_tag

        # Masking untuk special tokens ([CLS], [SEP], [PAD])
        final_labels = []
        for word_id, lbl in zip(word_ids, label_ids):
            if word_id is None:
                final_labels.append(-100)
            else:
                final_labels.append(lbl)

        labels.append(final_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## **Convert to HuggingFace Dataset & Split**

In [55]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_clean_train[[TEXT_COL, 'absa_labels_indexed']])

dataset = dataset.train_test_split(test_size=0.2, seed=42)

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

print("Contoh data training pertama:")
print(tokenized_datasets['train'][0]['input_ids'])
print(tokenized_datasets['train'][0]['labels'])

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Contoh data training pertama:
[3, 1836, 1994, 1540, 1731, 6356, 1485, 1608, 2855, 3218, 2588, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 11, 12, 12, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,

## **Training IndoBERT**

In [15]:
!pip install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=51e580ebbbb06101a3807595202b16a829f5b865ba00d7a0ad02cd3916b3ed38
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [56]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import evaluate

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [58]:
args = TrainingArguments(
    output_dir="./absa-indobert-results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10
)

In [62]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-3552743926.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.194600,1.315793,0.180077,0.271676,0.216590,0.610101
2,1.090600,1.202163,0.233051,0.317919,0.268949,0.646465
3,1.075900,1.152621,0.270386,0.364162,0.310345,0.660606
4,1.056000,1.133824,0.264069,0.352601,0.301980,0.655556
5,1.015900,1.131478,0.284483,0.381503,0.325926,0.660606


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=125, training_loss=1.1191713180541991, metrics={'train_runtime': 127.4055, 'train_samples_per_second': 15.541, 'train_steps_per_second': 0.981, 'total_flos': 129357096422400.0, 'train_loss': 1.1191713180541991, 'epoch': 5.0})

## **Inference**

In [63]:
from transformers import pipeline

checkpoint = "./absa-indobert-results/checkpoint-125"
token_classifier = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

Device set to use cuda:0


In [64]:
# Test 1 kalimat
text_sample = "Barangnya bagus tapi pengirimannya lama sekali."
results = token_classifier(text_sample)

print(f"Review: {text_sample}")
for res in results:
    print(f"Entity: {res['word']}, Label: {res['entity_group']}, Score: {res['score']:.4f}")

# --- Penjelasan Output ---
# Entity: bagus, Label: Kualitas_Barang
# Entity: lama sekali, Label: Pengiriman

Review: Barangnya bagus tapi pengirimannya lama sekali.
Entity: barangnya bagus tapi, Label: Kualitas_Barang, Score: 0.5903
Entity: pengirimannya lama sekali, Label: Pengiriman, Score: 0.4752
Entity: ., Label: Kualitas_Barang, Score: 0.2724


## **Load Model Sentimen & Aspek**

In [22]:
from transformers import pipeline

best_checkpoint = checkpoint

aspect_extractor = pipeline(
    "token-classification",
    model=best_checkpoint,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0
)

Device set to use cuda:0


In [23]:
sentiment_analyzer = pipeline(
    "text-classification",
    model="w11wo/indonesian-roberta-base-sentiment-classifier",
    device=0
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


In [24]:
from sklearn.metrics import classification_report

y_true_cat = []
y_pred_cat = []

for index, row in tqdm(df_clean_train.iterrows(), total=len(df_clean_train)):
    text = row[TEXT_COL]

    true_cats = set()
    for label in row['absa_labels']:
        if 'aspect' in label:
            true_cats.add(label['aspect'])

    try:
        results = aspect_extractor(text)
        pred_cats = set()
        for res in results:
            clean_cat = res['entity_group'].replace("_", " ")
            pred_cats.add(clean_cat)
    except:
        pred_cats = set()

    for aspect in ASPECT_LABELS:
        y_true_cat.append(1 if aspect in true_cats else 0)
        y_pred_cat.append(1 if aspect in pred_cats else 0)

print("\n=== HASIL EVALUASI INDOBERT (CATEGORY LEVEL) ===")
print(classification_report(y_true_cat, y_pred_cat, target_names=["Not Aspect", "Is Aspect"]))

  0%|          | 0/496 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



=== HASIL EVALUASI INDOBERT (CATEGORY LEVEL) ===
              precision    recall  f1-score   support

  Not Aspect       0.90      0.94      0.92      2575
   Is Aspect       0.79      0.70      0.74       897

    accuracy                           0.87      3472
   macro avg       0.85      0.82      0.83      3472
weighted avg       0.87      0.87      0.87      3472



## **Final Inference**

In [73]:
import json

final_results = []

for index, row in tqdm(df_test.iterrows(), total=len(df_test)):
    text = row[TEXT_COL]
    review_id = row.get('review_id', index)

    # Ekstraksi Aspek & Span
    try:
        aspect_results = aspect_extractor(text)
    except Exception as e:
        print(f"Error di review {index}: {e}")
        aspect_results = []

    absa_list = []

    # Untuk setiap aspek ditemukan, cari sentimennya
    for asp in aspect_results:
        span_text = asp['word']
        category = asp['entity_group']
        conf_score = asp['score']

        # Prediksi Sentimen dari span teks tersebut
        sent_result = sentiment_analyzer(span_text)[0]
        sentiment_label = sent_result['label']

        sent_map = {"positive": "pos", "negative": "neg", "neutral": "neu"}
        clean_sent = sent_map.get(sentiment_label, "neu")

        clean_category = category.replace("_", " ")

        absa_item = {
            "aspect": clean_category,
            "opinion_span": span_text,
            "sentiment": clean_sent,
            "confidence": float(f"{conf_score:.4f}"),
            "sent_id": 0
        }
        absa_list.append(absa_item)

    final_results.append({
        "review_id": review_id,
        "text": text,
        "absa_predict": json.dumps(absa_list)
    })

  0%|          | 0/1000 [00:00<?, ?it/s]

In [74]:
df_final = pd.DataFrame(final_results)

df_final.to_csv("absa_final_output.csv", index=False)

df_final.head()

,review_id,text,absa_predict
0,0,packing rapi mantap cepat sampai ke tujuan,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."
1,1,ya saya puas dengan barang,"[{""aspect"": ""Sesuai Deskripsi"", ""opinion_span""..."
2,2,respon luar biasa b mantap,"[{""aspect"": ""Pelayanan Penjual"", ""opinion_span..."
3,3,seller bagus pengiriman cepat barang oke,"[{""aspect"": ""Pelayanan Penjual"", ""opinion_span..."
4,4,pengiriman cepat seller bagus,"[{""aspect"": ""Pelayanan Penjual"", ""opinion_span..."


In [75]:
print("Contoh 5 review dari df_final:")
display(df_final.sample(5))

Contoh 5 review dari df_final:


,review_id,text,absa_predict
517,517,barang susah sampai packing rapi barang sesuai...,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."
61,61,barang sangat bagus sesuai deskripsi,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."
36,36,pesanan gx sesuai dengan yang d gambar,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."
621,621,barang telah diterima sesuai gambar dengan des...,"[{""aspect"": ""Pengiriman"", ""opinion_span"": ""bar..."
460,460,barang sesuai deskripsi dengan gambar semoga awet,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."


In [28]:
reviews_in_df_lab = df_lab['review_id'].tolist()

# Filter df_train to exclude reviews that are in df_lab
df_train_filtered = df_train[~df_train.index.isin(reviews_in_df_lab)].copy()

# Combine df_test and the filtered df_train for full inference
df_full_inference = pd.concat([df_test, df_train_filtered], ignore_index=True)
df_full_inference['review_id'] = df_full_inference.index

print(f"Shape of df_full_inference: {df_full_inference.shape}")
display(df_full_inference.head())

Shape of df_full_inference: (28468, 11)


,text,rating,category,product_name,product_id,sold,shop_id,product_url,text_clean,text_norm,review_id
0,Paket rapi...mantap....cepat....sampe ke tujuan,5,pertukangan,STAPLE GUN ATS 3 WAY TACKER - STAPLES JOK TEMB...,416032545,11,1477109,https://www.tokopedia.com/juraganperkakas/stap...,packing rapi mantap cepat sampai ke tujuan,packing rapi mantap cepat sampai ke tujuan,0
1,ya saya puas dgn barangnya,5,pertukangan,ALAT STAPLES TEMBAK &#40;AIR NAILER GUN&#41; O...,102279869,5,771395,https://www.tokopedia.com/kamarmesin/alat-stap...,ya saya puas dengan barang,ya saya puas dengan barang,1
2,Responya luar biasa b mantap,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,respon luar biasa b mantap,respon luar biasa b mantap,2
3,"seller top, pengiriman cepat barang oke",5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,seller bagus pengiriman cepat barang oke,seller bagus pengiriman cepat barang oke,3
4,pengiriman cepat seller top,5,pertukangan,Isi Refill Staples Jok Kulit Motor / Staple Gu...,190679689,787,969999,https://www.tokopedia.com/mitrapersada/isi-ref...,pengiriman cepat seller bagus,pengiriman cepat seller bagus,4


In [29]:
final_results_full = []

for index, row in tqdm(df_full_inference.iterrows(), total=len(df_full_inference)):
    text = row[TEXT_COL]
    review_id = row['review_id']

    # Ekstraksi Aspek & Span
    try:
        aspect_results = aspect_extractor(text)
    except Exception as e:
        print(f"Error di review {review_id}: {e}")
        aspect_results = []

    absa_list = []

    # Untuk setiap aspek ditemukan, cari sentimennya
    for asp in aspect_results:
        span_text = asp['word']
        category = asp['entity_group']
        conf_score = asp['score']

        # Prediksi Sentimen dari span teks tersebut
        sent_result = sentiment_analyzer(span_text)[0]
        sentiment_label = sent_result['label']

        sent_map = {"positive": "pos", "negative": "neg", "neutral": "neu"}
        clean_sent = sent_map.get(sentiment_label, "neu")

        clean_category = category.replace("_", " ")

        absa_item = {
            "aspect": clean_category,
            "opinion_span": span_text,
            "sentiment": clean_sent,
            "confidence": float(f"{conf_score:.4f}"),
            "sent_id": 0
        }
        absa_list.append(absa_item)

    final_results_full.append({
        "review_id": review_id,
        "text": text,
        "absa_predict": json.dumps(absa_list)
    })

  0%|          | 0/28468 [00:00<?, ?it/s]

In [30]:
df_final_full = pd.DataFrame(final_results_full)

df_final_full.to_csv("absa_full_inference_output.csv", index=False)

df_final_full.head()

,review_id,text,absa_predict
0,0,packing rapi mantap cepat sampai ke tujuan,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."
1,1,ya saya puas dengan barang,"[{""aspect"": ""Sesuai Deskripsi"", ""opinion_span""..."
2,2,respon luar biasa b mantap,"[{""aspect"": ""Pelayanan Penjual"", ""opinion_span..."
3,3,seller bagus pengiriman cepat barang oke,"[{""aspect"": ""Pelayanan Penjual"", ""opinion_span..."
4,4,pengiriman cepat seller bagus,"[{""aspect"": ""Pelayanan Penjual"", ""opinion_span..."


In [31]:
print("Contoh 5 review dari df_final_full:")
display(df_final_full.sample(5))

Contoh 5 review dari df_final_full:


,review_id,text,absa_predict
25880,25880,ggwp cepat datang rekomen toko,[]
15938,15938,penjual aktif d barang sampai di ramah barang ...,"[{""aspect"": ""Pengiriman"", ""opinion_span"": ""bar..."
6334,6334,sesuai dengan harga sip,"[{""aspect"": ""Sesuai Deskripsi"", ""opinion_span""..."
10901,10901,barang sesuai deskripsi pengiriman cepat hari ...,"[{""aspect"": ""Kualitas Barang"", ""opinion_span"":..."
26855,26855,sesuai barang yang dipesan size pas cepat respon,"[{""aspect"": ""Sesuai Deskripsi"", ""opinion_span""..."


## **Eksperimen: Joint Learning**

In [32]:
aspects = ASPECT_LABELS
sentiments = ["POS", "NEG", "NEU"]
joint_label_list = ["O"]

for asp in aspects:
    asp_code = asp.replace(" ", "_")
    for sent in sentiments:
        joint_label_list.append(f"B-{asp_code}-{sent}")
        joint_label_list.append(f"I-{asp_code}-{sent}")

joint_label2id = {l: i for i, l in enumerate(joint_label_list)}
joint_id2label = {i: l for i, l in enumerate(joint_label_list)}

print(f"Total Label Joint: {len(joint_label_list)}")
print("Label:", joint_label_list)

Total Label Joint: 43
Label: ['O', 'B-Kualitas_Barang-POS', 'I-Kualitas_Barang-POS', 'B-Kualitas_Barang-NEG', 'I-Kualitas_Barang-NEG', 'B-Kualitas_Barang-NEU', 'I-Kualitas_Barang-NEU', 'B-Pelayanan_Penjual-POS', 'I-Pelayanan_Penjual-POS', 'B-Pelayanan_Penjual-NEG', 'I-Pelayanan_Penjual-NEG', 'B-Pelayanan_Penjual-NEU', 'I-Pelayanan_Penjual-NEU', 'B-Kemasan_Barang-POS', 'I-Kemasan_Barang-POS', 'B-Kemasan_Barang-NEG', 'I-Kemasan_Barang-NEG', 'B-Kemasan_Barang-NEU', 'I-Kemasan_Barang-NEU', 'B-Harga_Barang-POS', 'I-Harga_Barang-POS', 'B-Harga_Barang-NEG', 'I-Harga_Barang-NEG', 'B-Harga_Barang-NEU', 'I-Harga_Barang-NEU', 'B-Sesuai_Deskripsi-POS', 'I-Sesuai_Deskripsi-POS', 'B-Sesuai_Deskripsi-NEG', 'I-Sesuai_Deskripsi-NEG', 'B-Sesuai_Deskripsi-NEU', 'I-Sesuai_Deskripsi-NEU', 'B-Pengiriman-POS', 'I-Pengiriman-POS', 'B-Pengiriman-NEG', 'I-Pengiriman-NEG', 'B-Pengiriman-NEU', 'I-Pengiriman-NEU', 'B-Lainnya-POS', 'I-Lainnya-POS', 'B-Lainnya-NEG', 'I-Lainnya-NEG', 'B-Lainnya-NEU', 'I-Lainnya-NEU']

In [33]:
def tokenize_and_align_joint_labels(examples):
    tokenized_inputs = tokenizer(
        examples[TEXT_COL],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=False
    )

    labels = []
    for i, text in enumerate(examples[TEXT_COL]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        original_labels = examples['absa_labels_indexed'][i]

        label_ids = [0] * len(tokenized_inputs["input_ids"][i])

        for lab in original_labels:
            start_char = lab['start']
            end_char = lab['end']

            cat_code = lab['aspect'].replace(" ", "_")

            raw_sent = lab.get('sentiment', 'neu').lower()
            if raw_sent == 'pos': sent_code = 'POS'
            elif raw_sent == 'neg': sent_code = 'NEG'
            else: sent_code = 'NEU'

            b_tag_str = f"B-{cat_code}-{sent_code}"
            i_tag_str = f"I-{cat_code}-{sent_code}"

            b_tag_id = joint_label2id.get(b_tag_str, 0)
            i_tag_id = joint_label2id.get(i_tag_str, 0)

            token_start = tokenized_inputs.char_to_token(i, start_char)
            token_end = tokenized_inputs.char_to_token(i, end_char - 1)

            if token_start is None or token_end is None: continue

            label_ids[token_start] = b_tag_id
            if token_end > token_start:
                for k in range(token_start + 1, token_end + 1):
                    label_ids[k] = i_tag_id

        final_labels = []
        for word_id, lbl in zip(word_ids, label_ids):
            if word_id is None: final_labels.append(-100)
            else: final_labels.append(lbl)
        labels.append(final_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [34]:
from datasets import Dataset

ds_joint = Dataset.from_pandas(df_clean_train[[TEXT_COL, 'absa_labels_indexed']])
ds_joint = ds_joint.train_test_split(test_size=0.2, seed=42)

tokenized_joint = ds_joint.map(tokenize_and_align_joint_labels, batched=True)

print(tokenized_joint['train'][0]['labels'])

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

[-100, 0, 0, 0, 0, 0, 0, 0, 31, 32, 32, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


In [35]:
from transformers import AutoModelForTokenClassification

model_joint = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=len(joint_label_list),
    id2label=joint_id2label,
    label2id=joint_label2id
)

args_joint = TrainingArguments(
    output_dir="./absa-joint-results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
def compute_metrics_joint(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [joint_label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [joint_label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
trainer_joint = Trainer(
    model=model_joint,
    args=args_joint,
    train_dataset=tokenized_joint["train"],
    eval_dataset=tokenized_joint["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_joint,
)

trainer_joint.train()

/tmp/ipython-input-3636076361.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_joint = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.212900,2.211373,0.000000,0.000000,0.000000,0.432323
2,2.067500,1.986308,0.107527,0.057803,0.075188,0.468687
3,1.973700,1.850026,0.138365,0.127168,0.132530,0.517172
4,1.884200,1.766672,0.183333,0.190751,0.186969,0.560606
5,1.781100,1.744026,0.179894,0.196532,0.187845,0.568687


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=125, training_loss=2.0681842498779295, metrics={'train_runtime': 272.5971, 'train_samples_per_second': 7.263, 'train_steps_per_second': 0.459, 'total_flos': 129389838842880.0, 'train_loss': 2.0681842498779295, 'epoch': 5.0})

In [38]:
from sklearn.metrics import classification_report
import numpy as np

predictions_output = trainer_joint.predict(tokenized_joint["test"])
preds_idx = np.argmax(predictions_output.predictions, axis=2)
labels_idx = predictions_output.label_ids

y_true_cat_joint = []
y_pred_cat_joint = []

for i in range(len(labels_idx)):
    true_tags = [joint_id2label[l] for l in labels_idx[i] if l != -100]
    pred_tags = [joint_id2label[p] for (p, l) in zip(preds_idx[i], labels_idx[i]) if l != -100]

    true_cats = set()
    for tag in true_tags:
        if tag != "O":
            parts = tag.split('-')
            if len(parts) >= 2:
                cat_name = parts[1].replace("_", " ")
                true_cats.add(cat_name)

    pred_cats = set()
    for tag in pred_tags:
        if tag != "O":
            parts = tag.split('-')
            if len(parts) >= 2:
                cat_name = parts[1].replace("_", " ")
                pred_cats.add(cat_name)

    for aspect in ASPECT_LABELS:
        y_true_cat_joint.append(1 if aspect in true_cats else 0)
        y_pred_cat_joint.append(1 if aspect in pred_cats else 0)

print("\n=== HASIL EVALUASI JOINT MODEL (CATEGORY LEVEL) ===")
print(classification_report(y_true_cat_joint, y_pred_cat_joint, target_names=["Not Aspect", "Is Aspect"]))

/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== HASIL EVALUASI JOINT MODEL (CATEGORY LEVEL) ===
              precision    recall  f1-score   support

  Not Aspect       0.92      0.94      0.93       530
   Is Aspect       0.80      0.74      0.77       170

    accuracy                           0.89       700
   macro avg       0.86      0.84      0.85       700
weighted avg       0.89      0.89      0.89       700

